In [ ]:
import time

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import MinMaxScaler
from torch.utils.data import DataLoader, TensorDataset

In [ ]:
# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")


In [ ]:
# Step 1: Load the dataset
file_path = "../../Combined_Data/combined_spar_data_full_parameters_split.csv"
df = pd.read_csv(file_path)

In [ ]:
# Step 2: Define features and labels
# Features - based on your provided images (excluding ib and ic which are now labels)
feature_columns = [
    "freq",  # Frequency
    "vb",
    "vc",  # Voltage parameters
    "DEV_GEOM_L",
    "NUM_OF_TRANS_RF",  # Device geometry
]

# Labels - de-embedded S-parameters

s_parameter_labels = [
    "S_deemb(1,1)_real",
    "S_deemb(1,1)_imag",
    "S_deemb(1,2)_real",
    "S_deemb(1,2)_imag",
    "S_deemb(2,1)_real",
    "S_deemb(2,1)_imag",
    "S_deemb(2,2)_real",
    "S_deemb(2,2)_imag",
]

In [ ]:
# Step 3: Check for null values in both features and labels
print("Checking for null values in features:")
feature_nulls = df[feature_columns].isnull().sum()
print(feature_nulls[feature_nulls > 0])  # Only show features with nulls

print("\nChecking for null values in labels:")
label_nulls = df[s_parameter_labels].isnull().sum()
print(label_nulls)

In [ ]:
# Step 4: Filter rows with any null values in features or labels
df_clean = df.dropna(subset=feature_columns + s_parameter_labels)

print(f"\nOriginal dataset shape: {df.shape}")
print(f"Cleaned dataset shape: {df_clean.shape}")
print(f"Removed {df.shape[0] - df_clean.shape[0]} rows with null values")

In [ ]:
# Step 5: Create separate dataframes for features and labels
X = df_clean[feature_columns].copy()
Y = df_clean[s_parameter_labels].copy()

# Print shapes to confirm
print(f"\nFeature dataset shape: {X.shape}")
print(f"S-parameter labels shape: {Y.shape}")

# Step 6: Basic statistics for all datasets
print("\nFeature statistics (first 5 columns):")
print(X.iloc[:, :5].describe())


print("\nS-parameter statistics (first 4 columns):")
print(Y.iloc[:, :4].describe())

# Optional: Save cleaned datasets to files
# X.to_csv("hbt_features.csv", index=False)
# Y.to_csv("hbt_sparam_labels.csv", index=False)

print("\nFeature and label separation complete!")

In [ ]:
def plot_feature_vs_label_correlations(X, y, target_names, filename):
    """Create a heatmap of correlations between features and labels"""
    # Calculate correlations
    combined = pd.concat([X, y], axis=1)
    correlation = combined.corr()

    # Extract only the correlations between features and labels
    feature_target_corr = correlation.loc[X.columns, target_names]

    # Plot heatmap
    plt.figure(figsize=(12, 10))
    sns.heatmap(feature_target_corr, annot=True, cmap="coolwarm", fmt=".2f")
    plt.title("Feature-Target Correlations")
    plt.tight_layout()
    plt.savefig(filename)
    plt.close()

    return feature_target_corr

In [ ]:
# Plot correlations for selected S-parameters (using just S11 as example)
s11_labels = ["S_deemb(1,1)_real", "S_deemb(1,1)_imag"]
s11_corr = plot_feature_vs_label_correlations(
    X, Y[s11_labels], s11_labels, "s11_correlations.png"
)
print("\nTop 5 features correlated with S11 parameters:")
for label in s11_labels:
    top_features = s11_corr[label].abs().sort_values(ascending=False).head(5)
    print(f"\nTop features for {label}:")
    print(top_features)

In [ ]:
# Plot correlations for selected S-parameters (using just S11 as example)
s12_labels = ["S_deemb(1,2)_real", "S_deemb(1,2)_imag"]
s12_corr = plot_feature_vs_label_correlations(
    X, Y[s12_labels], s12_labels, "s12_correlations.png"
)
print("\nTop 5 features correlated with S12 parameters:")
for label in s12_labels:
    top_features = s12_corr[label].abs().sort_values(ascending=False).head(5)
    print(f"\nTop features for {label}:")
    print(top_features)

In [ ]:
# Plot correlations for selected S-parameters (using just S11 as example)
s21_labels = ["S_deemb(2,1)_real", "S_deemb(2,1)_imag"]
s21_corr = plot_feature_vs_label_correlations(
    X, Y[s21_labels], s21_labels, "s21_correlations.png"
)
print("\nTop 5 features correlated with S21 parameters:")
for label in s21_labels:
    top_features = s21_corr[label].abs().sort_values(ascending=False).head(5)
    print(f"\nTop features for {label}:")
    print(top_features)

In [ ]:
# Plot correlations for selected S-parameters (using just S11 as example)
s22_labels = ["S_deemb(2,2)_real", "S_deemb(2,2)_imag"]
s22_corr = plot_feature_vs_label_correlations(
    X, Y[s22_labels], s22_labels, "s22_correlations.png"
)
print("\nTop 5 features correlated with S22 parameters:")
for label in s22_labels:
    top_features = s22_corr[label].abs().sort_values(ascending=False).head(5)
    print(f"\nTop features for {label}:")
    print(top_features)

In [ ]:
def create_extrapolation_split(
    df,
    train_threshold=40e9,
    test_freqs=[
        41,
        42,
        43,
        44,
        45,
        46,
        47,
        48,
        49,
        50,
        51,
        52,
        53,
        54,
        55,
        56,
        57,
        58,
        59,
        60,
        61,
        62,
        63,
        64,
        65,
    ],
):
    """
    Create a train-test split for extrapolation where:
    - Training set contains all data with frequency <= train_threshold
    - Test set contains specified frequencies above the threshold

    Parameters:
    -----------
    df : pandas DataFrame
        DataFrame containing a 'freq' column in Hz
    train_threshold : float, default=60e9
        Maximum frequency (in Hz) to include in training data
    test_freqs : list, default=[61, 62, 63, 64, 65]
        Specific frequencies (in GHz) to include in test set

    Returns:
    --------
    train_mask : numpy array
        Boolean mask for training data
    test_mask : numpy array
        Boolean mask for test data
    """
    import numpy as np

    # Convert test frequencies from GHz to Hz
    test_freqs_hz = np.array(test_freqs) * 1e9

    # Get all unique frequency values
    unique_freqs = np.sort(df["freq"].unique())

    # Create training mask: include all frequencies <= threshold
    train_mask = df["freq"] <= train_threshold

    # Create test mask: include only the specified test frequencies
    test_mask = df["freq"].isin(test_freqs_hz)

    # Print dataset information
    train_freqs = unique_freqs[unique_freqs <= train_threshold]
    test_freqs_found = np.intersect1d(unique_freqs, test_freqs_hz)
    test_freqs_missing = np.setdiff1d(test_freqs_hz, unique_freqs)

    print(
        f"Found {len(unique_freqs)} unique frequency values from {unique_freqs.min() / 1e9:.2f} GHz to {unique_freqs.max() / 1e9:.2f} GHz"
    )

    print(
        f"Training on {len(train_freqs)} unique frequencies from {train_freqs.min() / 1e9:.2f} GHz to {train_freqs.max() / 1e9:.2f} GHz"
    )
    print(f"Training set: {train_mask.sum()} samples")

    print(
        f"Testing on {len(test_freqs_found)} unique frequencies: {test_freqs_found / 1e9} GHz"
    )
    print(f"Test set: {test_mask.sum()} samples")

    if len(test_freqs_missing) > 0:
        print(
            f"Warning: {len(test_freqs_missing)} requested test frequencies not found in data: {test_freqs_missing / 1e9} GHz"
        )

    return train_mask, test_mask

In [ ]:
train_mask, test_mask = create_extrapolation_split(df)

# Use the masks to split features and labels
X_raw_train = X[train_mask].copy()
X_raw_test = X[test_mask].copy()
Y_raw_train = Y[train_mask].copy()
Y_raw_test = Y[test_mask].copy()

In [ ]:
# For training data
X_train = X_raw_train.copy()
X_train["vb_is_zero"] = (X_train["vb"] == 0).astype(int)
X_train["vb_is_high"] = ((X_train["vb"] >= 0.7) & (X_train["vb"] <= 0.9)).astype(int)
X_train["vc_is_zero"] = (X_train["vc"] == 0).astype(int)
X_train["vc_is_1_2V"] = ((X_train["vc"] >= 1.1) & (X_train["vc"] <= 1.3)).astype(int)
X_train["vc_is_1_5V"] = ((X_train["vc"] >= 1.4) & (X_train["vc"] <= 1.6)).astype(int)

# For test data
X_test = X_raw_test.copy()
X_test["vb_is_zero"] = (X_test["vb"] == 0).astype(int)
X_test["vb_is_high"] = ((X_test["vb"] >= 0.7) & (X_test["vb"] <= 0.9)).astype(int)
X_test["vc_is_zero"] = (X_test["vc"] == 0).astype(int)
X_test["vc_is_1_2V"] = ((X_test["vc"] >= 1.1) & (X_test["vc"] <= 1.3)).astype(int)
X_test["vc_is_1_5V"] = ((X_test["vc"] >= 1.4) & (X_test["vc"] <= 1.6)).astype(int)

# STEP 3: Initialize and fit scaler ONLY on training data
voltage_scaler = MinMaxScaler(feature_range=(-1, 1))
voltage_scaler.fit(X_train[["vb", "vc"]])  # Fit only on training data

# STEP 4: Transform both datasets using the fitted scaler
X_train[["vb", "vc"]] = voltage_scaler.transform(X_train[["vb", "vc"]])
X_test[["vb", "vc"]] = voltage_scaler.transform(X_test[["vb", "vc"]])

# STEP 5: Save the scaler for future use
import joblib

joblib.dump(voltage_scaler, "voltage_scaler.pkl")

In [ ]:
# Process training data
X_train = X_raw_train.copy()
X_train.loc[:, "DEV_L_0_9um"] = (X_train["DEV_GEOM_L"] == 0.9).astype(int)
X_train.loc[:, "DEV_L_2_5um"] = (X_train["DEV_GEOM_L"] == 2.5).astype(int)
X_train.loc[:, "DEV_L_5_0um"] = (X_train["DEV_GEOM_L"] == 5.0).astype(int)

# Drop the original column from training data
X_train = X_train.drop("DEV_GEOM_L", axis=1)

# Process test data with the same transformations
X_test = X_raw_test.copy()
X_test.loc[:, "DEV_L_0_9um"] = (X_test["DEV_GEOM_L"] == 0.9).astype(int)
X_test.loc[:, "DEV_L_2_5um"] = (X_test["DEV_GEOM_L"] == 2.5).astype(int)
X_test.loc[:, "DEV_L_5_0um"] = (X_test["DEV_GEOM_L"] == 5.0).astype(int)

# Drop the original column from test data
X_test = X_test.drop("DEV_GEOM_L", axis=1)

In [ ]:
# STEP 2: Process training data
X_train = X_raw_train.copy()
X_train.loc[:, "TRANS_1"] = (X_train["NUM_OF_TRANS_RF"] == 1).astype(int)
X_train.loc[:, "TRANS_2"] = (X_train["NUM_OF_TRANS_RF"] == 2).astype(int)
X_train.loc[:, "TRANS_4"] = (X_train["NUM_OF_TRANS_RF"] == 4).astype(int)

# Drop the original column from training data
X_train = X_train.drop("NUM_OF_TRANS_RF", axis=1)

# STEP 3: Process test data with the same transformations
X_test = X_raw_test.copy()
X_test.loc[:, "TRANS_1"] = (X_test["NUM_OF_TRANS_RF"] == 1).astype(int)
X_test.loc[:, "TRANS_2"] = (X_test["NUM_OF_TRANS_RF"] == 2).astype(int)
X_test.loc[:, "TRANS_4"] = (X_test["NUM_OF_TRANS_RF"] == 4).astype(int)

# Drop the original column from test data
X_test = X_test.drop("NUM_OF_TRANS_RF", axis=1)

In [ ]:
import importlib

import frequency_preprocessing

importlib.reload(frequency_preprocessing)
from frequency_preprocessing import preprocess_frequency

# Then try using it
X_train, X_test = preprocess_frequency(X_train, X_test, fit_mode=True)

In [ ]:
# First check if columns exist, create them if they don't
for i in range(1, 6):
    if f"freq_pos_in_band_{i}" not in X_train.columns:
        X_train[f"freq_pos_in_band_{i}"] = 0
    else:
        X_train[f"freq_pos_in_band_{i}"] = X_train[f"freq_pos_in_band_{i}"].fillna(0)

    if X_test is not None:
        if f"freq_pos_in_band_{i}" not in X_test.columns:
            X_test[f"freq_pos_in_band_{i}"] = 0
        else:
            X_test[f"freq_pos_in_band_{i}"] = X_test[f"freq_pos_in_band_{i}"].fillna(0)

# Fill any remaining NaN values in other columns
X_train = X_train.fillna(0)
if X_test is not None:
    X_test = X_test.fillna(0)

# ADD THIS CODE HERE - Make sure X_test has same columns in same order as X_train
X_test = X_test.reindex(columns=X_train.columns)
print("Column order match:", list(X_train.columns) == list(X_test.columns))

In [ ]:
import os

import pandas as pd
import torch
from sklearn.discriminant_analysis import StandardScaler

# Create directory for results
os.makedirs("freq_aware_results", exist_ok=True)


# Define SMAPE function for better handling of small values
def symmetric_mean_absolute_percentage_error(y_true, y_pred, epsilon=1e-10):
    """Calculate SMAPE with protection against division by zero."""
    denominator = (np.abs(y_true) + np.abs(y_pred)) / 2.0 + epsilon
    numerator = np.abs(y_true - y_pred)
    smape = numerator / denominator
    return np.mean(smape) * 100


# Define mean absolute percentage error function
def mean_absolute_percentage_error(y_true, y_pred, epsilon=1e-10):
    """Calculate MAPE with protection against division by zero."""
    non_zero = np.abs(y_true) > epsilon
    if non_zero.sum() == 0:
        return np.nan
    percentage_errors = (
        np.abs(
            (y_true[non_zero] - y_pred[non_zero]) / (np.abs(y_true[non_zero]) + epsilon)
        )
        * 100
    )
    return np.mean(percentage_errors)


# Frequency-aware neural network
class FrequencyAwareNetwork(nn.Module):
    def __init__(
        self,
        freq_features,
        other_features,
        hidden_sizes=[64, 128, 256],
        dropout_rate=0.2,
        activation="silu",
    ):
        super().__init__()

        if activation == "silu":
            activation_fn = nn.SiLU()
        elif activation == "relu":
            activation_fn = nn.ReLU()
        elif activation == "gelu":
            activation_fn = nn.GELU()
        else:
            raise ValueError(f"Unsupported activation function: {activation}")

        # Frequency-specific processing branch
        freq_layers = []
        prev_size = freq_features
        for h_size in hidden_sizes[:2]:  # First two hidden sizes for branches
            freq_layers.append(nn.Linear(prev_size, h_size))
            freq_layers.append(
                activation_fn
            )  # Using SiLU (Swish) activation for better performance
            freq_layers.append(nn.BatchNorm1d(h_size))
            freq_layers.append(nn.Dropout(dropout_rate))
            prev_size = h_size

        self.freq_branch = nn.Sequential(*freq_layers)

        # Other parameters branch
        other_layers = []
        prev_size = other_features
        for h_size in hidden_sizes[:2]:
            other_layers.append(nn.Linear(prev_size, h_size))
            other_layers.append(activation_fn)
            other_layers.append(nn.BatchNorm1d(h_size))
            other_layers.append(nn.Dropout(dropout_rate))
            prev_size = h_size

        self.other_branch = nn.Sequential(*other_layers)

        # Combined processing with residual connections
        combined_layers = []
        prev_size = hidden_sizes[1] * 2  # Output size from both branches combined

        for h_size in hidden_sizes[2:]:
            combined_layers.append(nn.Linear(prev_size, h_size))
            combined_layers.append(activation_fn)
            combined_layers.append(nn.BatchNorm1d(h_size))
            combined_layers.append(nn.Dropout(dropout_rate))
            prev_size = h_size

        # Final output layer for real and imaginary components
        combined_layers.append(nn.Linear(prev_size, 2))

        self.combined = nn.Sequential(*combined_layers)

        # Store feature indices for processing
        self.freq_indices = None
        self.other_indices = None

    def forward(self, x):
        # Split input into frequency and other features
        if self.freq_indices is None or self.other_indices is None:
            raise ValueError(
                "Feature indices not set. Call set_feature_indices() first."
            )

        freq_input = x[:, self.freq_indices]
        other_input = x[:, self.other_indices]

        # Process through branches
        freq_features = self.freq_branch(freq_input)
        other_features = self.other_branch(other_input)

        # Combine and output
        combined = torch.cat([freq_features, other_features], dim=1)
        return self.combined(combined)

    def set_feature_indices(self, freq_indices, other_indices):
        """Set indices for frequency and other features."""
        self.freq_indices = freq_indices
        self.other_indices = other_indices


# Helper function to identify frequency-related features
def identify_frequency_features(X_columns):
    """Identify frequency-related features in the dataset."""
    freq_features = [
        i
        for i, col in enumerate(X_columns)
        if "freq" in col.lower() or "band" in col.lower()
    ]
    other_features = [i for i in range(len(X_columns)) if i not in freq_features]

    print(
        f"Identified {len(freq_features)} frequency-related features and {len(other_features)} other features"
    )
    return freq_features, other_features


# Modified prepare_data_for_pytorch to handle scaling
def prepare_data_for_pytorch_with_scaling(
    X_train, Y_train, X_test, Y_test, components, batch_size=128, scale_y=True
):
    """Prepare data for PyTorch models with optional Y-scaling."""

    # Convert to PyTorch tensors
    X_train_tensor = torch.FloatTensor(X_train.values)
    X_test_tensor = torch.FloatTensor(X_test.values)

    # Handle Y data scaling if requested
    if scale_y:
        # Create scaler for Y values
        y_scaler = StandardScaler()
        Y_train_values = Y_train[components].values
        Y_test_values = Y_test[components].values

        # Fit scaler and transform data
        Y_train_scaled = y_scaler.fit_transform(Y_train_values)
        Y_test_scaled = y_scaler.transform(Y_test_values)

        # Convert to tensors
        Y_train_tensor = torch.FloatTensor(Y_train_scaled)
        Y_test_tensor = torch.FloatTensor(Y_test_scaled)

        # Save scaler for later use
        component_str = "_".join(components)
        joblib.dump(y_scaler, f"freq_aware_results/{component_str}_scaler.pkl")

        # Create data loaders
        train_dataset = TensorDataset(X_train_tensor, Y_train_tensor)
        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

        return (
            X_train_tensor,
            Y_train_tensor,
            X_test_tensor,
            Y_test_tensor,
            train_loader,
            y_scaler,
        )

    else:
        # No scaling
        Y_train_tensor = torch.FloatTensor(Y_train[components].values)
        Y_test_tensor = torch.FloatTensor(Y_test[components].values)

        # Create data loaders
        train_dataset = TensorDataset(X_train_tensor, Y_train_tensor)
        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

        return (
            X_train_tensor,
            Y_train_tensor,
            X_test_tensor,
            Y_test_tensor,
            train_loader,
            None,
        )


def train_model(
    model,
    train_loader,
    X_test_tensor,
    Y_test_tensor,
    criterion,
    optimizer,
    device,
    epochs=100,
    early_stopping_patience=15,
    verbose=True,
    lr_scheduler_type="reduce_on_plateau",
    warmup_epochs=5,
):
    """Train a PyTorch model with early stopping and learning rate scheduling."""
    model = model.to(device)

    # Set up learning rate scheduler based on specified type
    if lr_scheduler_type == "reduce_on_plateau":
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
            optimizer, mode="min", factor=0.85, patience=5, verbose=verbose, min_lr=5e-7
        )
    elif lr_scheduler_type == "cosine_annealing":
        scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
            optimizer, T_max=epochs, eta_min=1e-6
        )
    elif lr_scheduler_type == "one_cycle":
        scheduler = torch.optim.lr_scheduler.OneCycleLR(
            optimizer,
            max_lr=optimizer.param_groups[0]["lr"],
            steps_per_epoch=len(train_loader),
            epochs=epochs,
        )
    else:
        scheduler = None

    # For early stopping
    best_loss = float("inf")
    best_model_state = None
    patience_counter = 0

    # Track losses and learning rates for plotting
    train_losses = []
    val_losses = []
    learning_rates = []

    # Training loop
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0

        # Apply learning rate warmup if needed
        if warmup_epochs > 0 and epoch < warmup_epochs and scheduler is None:
            lr_multiplier = (epoch + 1) / warmup_epochs
            for param_group in optimizer.param_groups:
                param_group["lr"] = optimizer.param_groups[0]["lr"] * lr_multiplier

        # Record current learning rate
        current_lr = optimizer.param_groups[0]["lr"]
        learning_rates.append(current_lr)

        for inputs, targets in train_loader:
            inputs, targets = inputs.to(device), targets.to(device)

            # Forward pass
            outputs = model(inputs)
            loss = criterion(outputs, targets)

            # Backward and optimize
            optimizer.zero_grad()
            loss.backward()

            # Gradient clipping
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

            optimizer.step()

            # Step OneCycleLR scheduler here if being used
            if lr_scheduler_type == "one_cycle":
                scheduler.step()

            running_loss += loss.item()

        # Calculate average training loss
        avg_train_loss = running_loss / len(train_loader)
        train_losses.append(avg_train_loss)

        # Validation loss
        model.eval()
        with torch.no_grad():
            val_outputs = model(X_test_tensor.to(device))
            val_loss = criterion(val_outputs, Y_test_tensor.to(device)).item()
            val_losses.append(val_loss)

        # Print progress
        if verbose and (epoch + 1) % 10 == 0:
            print(
                f"Epoch {epoch + 1}/{epochs}, Train Loss: {avg_train_loss:.6f}, Val Loss: {val_loss:.6f}, LR: {current_lr:.8f}"
            )

        # Learning rate scheduler step (except for OneCycleLR which is done per iteration)
        if scheduler is not None:
            if lr_scheduler_type == "reduce_on_plateau":
                scheduler.step(val_loss)
            elif lr_scheduler_type == "cosine_annealing":
                scheduler.step()

        # Check for early stopping
        if val_loss < best_loss:
            best_loss = val_loss
            best_model_state = model.state_dict().copy()
            patience_counter = 0
        else:
            patience_counter += 1
            if patience_counter >= early_stopping_patience:
                if verbose:
                    print(f"Early stopping at epoch {epoch + 1}")
                break

    # Load best model
    if best_model_state is not None:
        model.load_state_dict(best_model_state)

    # Plot learning rate schedule
    plt.figure(figsize=(10, 4))
    plt.plot(learning_rates)
    plt.xlabel("Epochs")
    plt.ylabel("Learning Rate")
    plt.title("Learning Rate Schedule")
    plt.yscale("log")
    plt.savefig("freq_aware_results/learning_rate_schedule.png")
    plt.close()

    return model, train_losses, val_losses


# Modified evaluate_model function to handle scaling
def evaluate_model_with_scaling(
    model, X_test_tensor, Y_test_tensor, Y_test, components, device, y_scaler=None
):
    """Evaluate a trained model and calculate performance metrics."""
    model.eval()
    with torch.no_grad():
        predictions = model(X_test_tensor.to(device)).cpu().numpy()

    # Inverse transform if scaler was used
    if y_scaler is not None:
        predictions_original = y_scaler.inverse_transform(predictions)
        y_test_original = Y_test[components].values
    else:
        predictions_original = predictions
        y_test_original = Y_test[components].values

    # Calculate metrics
    metrics = {}

    for i, component in enumerate(components):
        y_true = y_test_original[:, i]
        y_pred = predictions_original[:, i]

        mse = mean_squared_error(y_true, y_pred)
        rmse = np.sqrt(mse)
        r2 = r2_score(y_true, y_pred)
        mae = mean_absolute_error(y_true, y_pred)

        # Use SMAPE instead of MAPE for S12
        if "S12" in component or "S_deemb(1,2)" in component:
            smape_val = symmetric_mean_absolute_percentage_error(y_true, y_pred)
            metrics[component] = {
                "mse": mse,
                "rmse": rmse,
                "r2": r2,
                "mae": mae,
                "smape": smape_val,
            }
        else:
            # Regular MAPE for other S-parameters
            metrics[component] = {
                "mse": mse,
                "rmse": rmse,
                "r2": r2,
                "mae": mae,
                "mape": mean_absolute_percentage_error(y_true, y_pred),
            }

    # Calculate average metrics
    avg_metrics = {
        "rmse": np.mean([metrics[comp]["rmse"] for comp in components]),
        "r2": np.mean([metrics[comp]["r2"] for comp in components]),
        "mae": np.mean([metrics[comp]["mae"] for comp in components]),
    }

    # Add SMAPE or MAPE average depending on which components were evaluated
    if any("S12" in comp or "S_deemb(1,2)" in comp for comp in components):
        avg_metrics["smape"] = np.mean([metrics[comp]["smape"] for comp in components])
    else:
        avg_metrics["mape"] = np.mean([metrics[comp]["mape"] for comp in components])

    return metrics, avg_metrics, predictions_original


def plot_learning_curves(train_losses, val_losses, model_name):
    """Plot the learning curves."""
    plt.figure(figsize=(10, 6))
    plt.plot(train_losses, label="Training Loss")
    plt.plot(val_losses, label="Validation Loss")
    plt.xlabel("Epochs")
    plt.ylabel("Loss")
    plt.title(f"Learning Curves for {model_name}")
    plt.legend()
    plt.savefig(f"freq_aware_results/learning_curves_{model_name}.png")
    plt.close()


def plot_predictions(Y_test, predictions, components, model_name):
    """Plot predictions vs actual values."""
    fig, axes = plt.subplots(1, len(components), figsize=(15, 5))

    for i, component in enumerate(components):
        ax = axes[i] if len(components) > 1 else axes
        y_true = Y_test[component].values
        y_pred = predictions[:, i]

        ax.scatter(y_true, y_pred, alpha=0.3)
        ax.plot([min(y_true), max(y_true)], [min(y_true), max(y_true)], "r--")
        ax.set_xlabel("Actual")
        ax.set_ylabel("Predicted")
        ax.set_title(f"{component}")

    plt.tight_layout()
    plt.savefig(f"freq_aware_results/predictions_{model_name}.png")
    plt.close()


def plot_error_distribution(Y_test, predictions, components, model_name):
    """Plot error distributions."""
    fig, axes = plt.subplots(1, len(components), figsize=(15, 5))

    for i, component in enumerate(components):
        ax = axes[i] if len(components) > 1 else axes
        y_true = Y_test[component].values
        y_pred = predictions[:, i]

        errors = y_pred - y_true

        sns.histplot(errors, kde=True, ax=ax)
        ax.set_xlabel("Prediction Error")
        ax.set_ylabel("Frequency")
        ax.set_title(f"{component} Error Distribution")

    plt.tight_layout()
    plt.savefig(f"freq_aware_results/error_dist_{model_name}.png")
    plt.close()


# Modified train_frequency_aware_models function
def train_frequency_aware_models(
    X_train, X_test, Y_train, Y_test, hyperparameters=None, selected_features=None
):
    """
    Train frequency-aware models for each S-parameter with conditional scaling.
    """
    # S-parameter definitions
    s_parameter_models = {"S22": ["S_deemb(2,2)_real", "S_deemb(2,2)_imag"]}

    # 'S12': ['S_deemb(1,2)_real', 'S_deemb(1,2)_imag']

    # Set default hyperparameters if not provided
    if hyperparameters is None:
        hyperparameters = {
            "hidden_sizes": [64, 128, 256],
            "dropout_rate": 0.2,
            "learning_rate": 0.001,
            "batch_size": 256,
            "epochs": 150,
            "early_stopping_patience": 15,
            "activation": "gelu",
            "lr_scheduler_type": "one_cycle",
        }

    # Filter features if requested
    if selected_features is not None:
        X_train = X_train[selected_features]
        X_test = X_test[selected_features]
        print(f"Using {len(selected_features)} selected features")

    # Check for GPU
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")

    # Identify frequency-related features
    freq_indices, other_indices = identify_frequency_features(X_train.columns)

    # Store results and models
    models = {}
    all_results = {}
    all_predictions = {}
    scalers = {}  # Store scalers for each model

    # Record start time
    start_time = time.time()

    # Train a model for each S-parameter
    for model_name, components in s_parameter_models.items():
        print(f"\n{'=' * 50}")
        print(f"Training frequency-aware model for {model_name}")
        print(f"{'=' * 50}")

        # Decide whether to scale Y data (only for S12)
        scale_y = model_name == "S12" or model_name == "S21"

        # Prepare data with conditional scaling
        prep_results = prepare_data_for_pytorch_with_scaling(
            X_train,
            Y_train,
            X_test,
            Y_test,
            components,
            hyperparameters["batch_size"],
            scale_y=scale_y,
        )

        if scale_y:
            (
                X_train_tensor,
                Y_train_tensor,
                X_test_tensor,
                Y_test_tensor,
                train_loader,
                y_scaler,
            ) = prep_results
            scalers[model_name] = y_scaler
            print("Applied StandardScaler to Y values for S12")
        else:
            (
                X_train_tensor,
                Y_train_tensor,
                X_test_tensor,
                Y_test_tensor,
                train_loader,
                _,
            ) = prep_results

        # Initialize model
        model = FrequencyAwareNetwork(
            len(freq_indices),
            len(other_indices),
            hyperparameters["hidden_sizes"],
            hyperparameters["dropout_rate"],
            hyperparameters.get("activation", "gelu"),
        )
        model.set_feature_indices(freq_indices, other_indices)

        # Loss and optimizer
        # Loss and optimizer - Use Huber Loss for S21 to handle outliers better
        if model_name == "S21":
            criterion = nn.HuberLoss(delta=0.1)  # Less sensitive to outliers
            print("Using Huber Loss for S21")
        else:
            criterion = nn.MSELoss()
        optimizer = optim.Adam(model.parameters(), lr=hyperparameters["learning_rate"])

        # Train model (use your existing train_model function)
        trained_model, train_losses, val_losses = train_model(
            model,
            train_loader,
            X_test_tensor,
            Y_test_tensor,
            criterion,
            optimizer,
            device,
            hyperparameters["epochs"],
            hyperparameters["early_stopping_patience"],
            lr_scheduler_type=hyperparameters.get("lr_scheduler_type", "one_cycle"),
        )

        # Plot learning curves
        plot_learning_curves(train_losses, val_losses, model_name)

        # Evaluate model with proper scaling handling
        metrics, avg_metrics, predictions = evaluate_model_with_scaling(
            trained_model,
            X_test_tensor,
            Y_test_tensor,
            Y_test,
            components,
            device,
            scalers.get(model_name),
        )

        # Plot predictions and error distributions
        plot_predictions(Y_test, predictions, components, model_name)
        plot_error_distribution(Y_test, predictions, components, model_name)

        # Print results
        print(f"\nPerformance metrics for {model_name}:")
        for component, metric in metrics.items():
            print(f"  {component}:")
            print(f"    RMSE: {metric['rmse']:.6f}")
            print(f"    R²: {metric['r2']:.6f}")
            print(f"    MAE: {metric['mae']:.6f}")
            if "smape" in metric:
                print(f"    SMAPE: {metric['smape']:.2f}%")
            else:
                print(f"    MAPE: {metric['mape']:.2f}%")

        print(f"\nAverage metrics for {model_name}:")
        print(f"  R²: {avg_metrics['r2']:.6f}")
        print(f"  RMSE: {avg_metrics['rmse']:.6f}")
        print(f"  MAE: {avg_metrics['mae']:.6f}")
        if "smape" in avg_metrics:
            print(f"  SMAPE: {avg_metrics['smape']:.2f}%")
        else:
            print(f"  MAPE: {avg_metrics['mape']:.2f}%")

        # Store results
        models[model_name] = trained_model
        all_results[model_name] = {
            "component_metrics": metrics,
            "avg_metrics": avg_metrics,
        }
        all_predictions[model_name] = predictions

    # Record total training time
    train_time = time.time() - start_time
    print(f"\nTotal training time: {train_time:.2f} seconds")

    # Save models
    for model_name, model in models.items():
        torch.save(model.state_dict(), f"freq_aware_results/{model_name}_model.pth")

    print("Models and results saved to freq_aware_results/")

    return models, all_results, all_predictions, scalers


# Function to experiment with different hyperparameters
def hyperparameter_tuning(X_train, X_test, Y_train, Y_test, param_grid):
    """
    Perform hyperparameter tuning by training models with different configurations.

    Parameters:
    -----------
    X_train, X_test : pd.DataFrame
        Preprocessed feature datasets
    Y_train, Y_test : pd.DataFrame
        Target S-parameter datasets
    param_grid : dict
        Dictionary of hyperparameter values to try

    Returns:
    --------
    results : dict
        Dictionary of results for each configuration
    """
    results = {}

    # Generate all hyperparameter combinations
    param_keys = list(param_grid.keys())
    param_values = list(param_grid.values())

    def generate_combinations(index, current_params):
        if index == len(param_keys):
            # Train model with current parameter combination
            config_name = "_".join([f"{k}={v}" for k, v in current_params.items()])
            print(f"\n\n{'#' * 70}")
            print(f"# Testing configuration: {config_name}")
            print(f"{'#' * 70}\n")

            # Train models
            _, all_results, _ = train_frequency_aware_models(
                X_train, X_test, Y_train, Y_test, hyperparameters=current_params
            )

            # Store results
            avg_r2 = np.mean(
                [result["avg_metrics"]["r2"] for result in all_results.values()]
            )
            results[config_name] = {
                "params": current_params.copy(),
                "avg_r2": avg_r2,
                "detailed_results": all_results,
            }
            return

        # Recursive exploration of parameter combinations
        for value in param_values[index]:
            current_params[param_keys[index]] = value
            generate_combinations(index + 1, current_params)

    # Start generating combinations
    generate_combinations(0, {})

    # Rank results
    ranked_results = sorted(results.items(), key=lambda x: x[1]["avg_r2"], reverse=True)

    # Print summary
    print("\n\n" + "=" * 80)
    print("HYPERPARAMETER TUNING RESULTS")
    print("=" * 80)

    for i, (config_name, result) in enumerate(ranked_results):
        print(f"\n{i + 1}. Configuration: {config_name}")
        print(f"   Average R²: {result['avg_r2']:.6f}")
        print(f"   Parameters: {result['params']}")

    return results


# Function to test different feature subsets
def feature_selection_experiment(X_train, X_test, Y_train, Y_test, feature_sets):
    """
    Test different feature subsets to find optimal combinations.

    Parameters:
    -----------
    X_train, X_test : pd.DataFrame
        Complete feature datasets
    Y_train, Y_test : pd.DataFrame
        Target S-parameter datasets
    feature_sets : dict
        Dictionary mapping set names to lists of feature columns

    Returns:
    --------
    results : dict
        Dictionary of results for each feature set
    """
    results = {}

    for set_name, features in feature_sets.items():
        print(f"\n\n{'#' * 70}")
        print(f"# Testing feature set: {set_name} ({len(features)} features)")
        print(f"{'#' * 70}\n")

        # Train models with this feature set
        _, all_results, _ = train_frequency_aware_models(
            X_train, X_test, Y_train, Y_test, selected_features=features
        )

        # Store results
        avg_r2 = np.mean(
            [result["avg_metrics"]["r2"] for result in all_results.values()]
        )
        results[set_name] = {
            "features": features,
            "feature_count": len(features),
            "avg_r2": avg_r2,
            "detailed_results": all_results,
        }

    # Rank results
    ranked_results = sorted(results.items(), key=lambda x: x[1]["avg_r2"], reverse=True)

    # Print summary
    print("\n\n" + "=" * 80)
    print("FEATURE SELECTION RESULTS")
    print("=" * 80)

    for i, (set_name, result) in enumerate(ranked_results):
        print(f"\n{i + 1}. Feature Set: {set_name}")
        print(f"   Features: {len(result['features'])}")
        print(f"   Average R²: {result['avg_r2']:.6f}")

    return results


# Example usage


# Example of running with all features and default hyperparameters
# models, results, predictions = train_frequency_aware_models(
#     X_train, X_test, Y_raw_train, Y_raw_test,
#     hyperparameters=default_hyperparameters
# )

# Example of hyperparameter tuning
# param_grid = {
#     'learning_rate': [0.0001, 0.001, 0.01],
#     'dropout_rate': [0.1, 0.2, 0.3],
#     'batch_size': [128, 256, 512]
# }
# tuning_results = hyperparameter_tuning(X_train, X_test, Y_raw_train, Y_raw_test, param_grid)

# Example of feature selection experiment
# core_features = ['freq', 'vb', 'vc', 'gm_abs_log']
# freq_features = [col for col in X_train.columns if 'freq' in col]
# impedance_features = [col for col in X_train.columns if 'Zin' in col or 'Zout' in col]

# feature_sets = {
#     'all_features': X_train.columns.tolist(),
#     'frequency_only': freq_features,
#     'core_plus_frequency': core_features + freq_features,
#     'core_plus_impedance': core_features + impedance_features,
#     'optimized_set': ['freq', 'freq_log', 'freq_log_norm', 'vb', 'vc', 'gm_abs_log',
#                       'Zin_real_log', 'Zin_imag_log', 'Zout_real_log']
# }
# feature_results = feature_selection_experiment(X_train, X_test, Y_raw_train, Y_raw_test, feature_sets)

## For Training S(2,1):


In [ ]:
def train_frequency_aware_models(
    X_train, X_test, Y_train, Y_test, hyperparameters=None, selected_features=None
):
    """
    Train frequency-aware models for each S-parameter with separate models for real/imaginary,
    and different architectures for frequency vs other parameters.
    """
    # S-parameter definitions
    s_parameter_components = {
        "S21_real": "S_deemb(2,1)_real",
        "S21_imag": "S_deemb(2,1)_imag",
    }

    # Set default hyperparameters if not provided
    if hyperparameters is None:
        hyperparameters = {
            "hidden_sizes": [64, 128, 256],
            "dropout_rate": 0.2,
            "learning_rate": 0.001,
            "batch_size": 256,
            "epochs": 150,
            "early_stopping_patience": 15,
            "activation": "gelu",
            "lr_scheduler_type": "one_cycle",
            "weight_decay": 1e-5,
        }

    # Extract architecture-specific parameters if provided
    freq_hidden_sizes = hyperparameters.get("freq_hidden_sizes", [64, 128])
    other_hidden_sizes = hyperparameters.get("other_hidden_sizes", [64, 128])

    # Use component-specific network sizes if provided
    real_hidden_sizes = hyperparameters.get(
        "real_hidden_sizes", hyperparameters["hidden_sizes"]
    )
    imag_hidden_sizes = hyperparameters.get(
        "imag_hidden_sizes", hyperparameters["hidden_sizes"]
    )

    # Filter features if requested
    if selected_features is not None:
        X_train = X_train[selected_features]
        X_test = X_test[selected_features]
        print(f"Using {len(selected_features)} selected features")

    # Check for GPU
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")

    # Create a copy of data and add normalized frequency for better extrapolation
    X_train_enhanced = X_train.copy()
    X_test_enhanced = X_test.copy()

    # Add normalized frequency (critical for extrapolation)
    max_freq = X_train["freq"].max()
    X_train_enhanced["freq_ratio"] = X_train["freq"] / max_freq
    X_test_enhanced["freq_ratio"] = X_test["freq"] / max_freq

    # Also add inverse freq to capture low-frequency behavior
    X_train_enhanced["freq_inv"] = 1.0 / (
        X_train["freq"] + 1e5
    )  # Avoid division by zero
    X_test_enhanced["freq_inv"] = 1.0 / (X_test["freq"] + 1e5)

    # Add log frequency to better capture wide frequency ranges
    X_train_enhanced["freq_log"] = np.log(X_train["freq"] + 1e5)
    X_test_enhanced["freq_log"] = np.log(X_test["freq"] + 1e5)

    # Add squared frequency for high-frequency behavior
    X_train_enhanced["freq_squared"] = (X_train["freq"] / max_freq) ** 2
    X_test_enhanced["freq_squared"] = (X_test["freq"] / max_freq) ** 2

    # Analyze S21 real/imag training data
    s21_real_train = Y_train["S_deemb(2,1)_real"].values
    s21_imag_train = Y_train["S_deemb(2,1)_imag"].values

    print(
        f"S21 real training range: {s21_real_train.min():.6f} to {s21_real_train.max():.6f}"
    )
    print(
        f"S21 imag training range: {s21_imag_train.min():.6f} to {s21_imag_train.max():.6f}"
    )

    # Get high-frequency training statistics
    X_train_s21 = X_train.copy()
    X_train_s21["S21_real"] = s21_real_train
    X_train_s21["S21_imag"] = s21_imag_train

    high_freq_threshold = np.percentile(X_train_s21["freq"], 80)
    high_freq_data = X_train_s21[X_train_s21["freq"] >= high_freq_threshold]

    print(
        f"Using high-frequency training data (>{high_freq_threshold / 1e9:.1f} GHz) for stabilization"
    )

    high_freq_real_mean = high_freq_data["S21_real"].mean()
    high_freq_imag_mean = high_freq_data["S21_imag"].mean()

    print(
        f"High-frequency S21 means: real={high_freq_real_mean:.6f}, imag={high_freq_imag_mean:.6f}"
    )

    # Calculate different bounds for real and imaginary parts
    # For real part - tighter bounds due to problems with this component
    real_p10 = np.percentile(s21_real_train, 10)
    real_p90 = np.percentile(s21_real_train, 90)
    real_range = real_p90 - real_p10
    real_min = real_p10 - 0.2 * real_range  # Tighter bound for real
    real_max = real_p90 + 0.2 * real_range

    # For imaginary part - more relaxed bounds since it's behaving better
    imag_p05 = np.percentile(s21_imag_train, 5)
    imag_p95 = np.percentile(s21_imag_train, 95)
    imag_range = imag_p95 - imag_p05
    imag_min = imag_p05 - 0.3 * imag_range  # More relaxed bound
    imag_max = imag_p95 + 0.3 * imag_range

    print("Setting component-specific bounds:")
    print(f"  Real: [{real_min:.6f}, {real_max:.6f}]")
    print(f"  Imaginary: [{imag_min:.6f}, {imag_max:.6f}]")

    # Identify frequency-related features
    freq_indices, other_indices = identify_frequency_features(X_train_enhanced.columns)

    # Store results and models
    models = {}
    all_results = {}
    all_predictions = {}
    scalers = {}  # Store scalers for each model

    # Record start time
    start_time = time.time()

    # Final predictions
    s21_predictions = np.zeros((len(X_test), 2))

    # Log-cosh loss function for robust training
    def log_cosh_loss(y_pred, y_true):
        return torch.mean(torch.log(torch.cosh(y_pred - y_true)))

    # Train a separate model for each component
    for model_name, component in s_parameter_components.items():
        print(f"\n{'=' * 50}")
        print(f"Training model for {model_name}")
        print(f"{'=' * 50}")

        # Component-specific hyperparameters
        if model_name == "S21_real":
            # More aggressive regularization for real component
            comp_hyperparams = hyperparameters.copy()
            comp_hyperparams["dropout_rate"] = (
                0.3  # Higher dropout but not as aggressive
            )
            comp_hyperparams["learning_rate"] = 0.001  # Lower learning rate
            comp_hyperparams["weight_decay"] = (
                hyperparameters.get("weight_decay", 1e-5) * 2
            )  # Higher L2 regularization
            comp_hidden_sizes = real_hidden_sizes
            scheduler_type = hyperparameters.get("real_scheduler", "cosine_annealing")
        else:
            # Standard hyperparameters for imaginary component
            comp_hyperparams = hyperparameters.copy()
            comp_hyperparams["dropout_rate"] = 0.2
            comp_hyperparams["weight_decay"] = hyperparameters.get("weight_decay", 1e-5)
            comp_hidden_sizes = imag_hidden_sizes
            scheduler_type = hyperparameters.get("imag_scheduler", "reduce_on_plateau")

        # Prepare data for this component
        Y_train_comp = Y_train[[component]].copy()
        Y_test_comp = Y_test[[component]].copy()

        # Scale inputs and outputs
        x_scaler = StandardScaler()
        y_scaler = StandardScaler()

        # Store the scalers
        scalers[model_name] = {"x": x_scaler, "y": y_scaler}

        # Scale X data - only scale non-frequency features
        X_train_scaled = X_train_enhanced.copy()
        X_test_scaled = X_test_enhanced.copy()

        # Get non-frequency columns to scale
        non_freq_cols = [
            col for col in X_train_enhanced.columns if "freq" not in col.lower()
        ]

        # Scale those columns
        X_train_scaled[non_freq_cols] = x_scaler.fit_transform(
            X_train_enhanced[non_freq_cols]
        )
        X_test_scaled[non_freq_cols] = x_scaler.transform(
            X_test_enhanced[non_freq_cols]
        )

        # Scale Y data
        Y_train_scaled = y_scaler.fit_transform(Y_train_comp)

        # Convert to tensors
        X_train_tensor = torch.FloatTensor(X_train_scaled.values).to(device)
        Y_train_tensor = torch.FloatTensor(Y_train_scaled).to(device)
        X_test_tensor = torch.FloatTensor(X_test_scaled.values).to(device)

        # Create data loader
        train_dataset = TensorDataset(X_train_tensor, Y_train_tensor)
        train_loader = DataLoader(
            train_dataset, batch_size=comp_hyperparams["batch_size"], shuffle=True
        )

        # Define a focused model specifically for this component with different branch architectures
        class ComponentModel(nn.Module):
            def __init__(
                self,
                input_size,
                hidden_sizes,
                dropout_rate,
                freq_indices,
                other_indices,
            ):
                super().__init__()
                self.freq_indices = freq_indices
                self.other_indices = other_indices

                # Frequency branch with custom architecture
                freq_layers = []
                input_size_freq = len(freq_indices)
                for i in range(len(freq_hidden_sizes)):
                    out_size = freq_hidden_sizes[i]
                    freq_layers.append(nn.Linear(input_size_freq, out_size))
                    freq_layers.append(nn.BatchNorm1d(out_size))
                    freq_layers.append(nn.GELU())
                    freq_layers.append(nn.Dropout(dropout_rate))
                    input_size_freq = out_size
                self.freq_layers = nn.Sequential(*freq_layers)

                # Other parameters branch with custom architecture
                other_layers = []
                input_size_other = len(other_indices)
                for i in range(len(other_hidden_sizes)):
                    out_size = other_hidden_sizes[i]
                    other_layers.append(nn.Linear(input_size_other, out_size))
                    other_layers.append(nn.BatchNorm1d(out_size))
                    other_layers.append(nn.GELU())
                    other_layers.append(nn.Dropout(dropout_rate))
                    input_size_other = out_size
                self.other_layers = nn.Sequential(*other_layers)

                # Attention mechanism for better integration of branches
                self.attention = nn.Sequential(
                    nn.Linear(freq_hidden_sizes[-1] + other_hidden_sizes[-1], 64),
                    nn.GELU(),
                    nn.Linear(64, 2),
                    nn.Softmax(dim=1),
                )

                # Combined layers
                combined_size = freq_hidden_sizes[-1] + other_hidden_sizes[-1]

                combined_layers = []
                input_size_combined = combined_size
                for i in range(len(hidden_sizes) - 1):
                    combined_layers.append(
                        nn.Linear(input_size_combined, hidden_sizes[i])
                    )
                    combined_layers.append(nn.BatchNorm1d(hidden_sizes[i]))
                    combined_layers.append(nn.GELU())
                    combined_layers.append(nn.Dropout(dropout_rate))
                    input_size_combined = hidden_sizes[i]

                # Output layer
                combined_layers.append(
                    nn.Linear(
                        hidden_sizes[-2]
                        if len(hidden_sizes) > 1
                        else input_size_combined,
                        1,
                    )
                )

                # Apply tanh only to real component to constrain outputs
                if model_name == "S21_real":
                    combined_layers.append(nn.Tanh())

                self.combined_layers = nn.Sequential(*combined_layers)

            def forward(self, x):
                # Extract frequency and other inputs
                freq_input = x[:, self.freq_indices]
                other_input = x[:, self.other_indices]

                # Process through respective branches
                freq_features = self.freq_layers(freq_input)
                other_features = self.other_layers(other_input)

                # Combine features
                combined = torch.cat([freq_features, other_features], dim=1)

                # Apply attention mechanism
                attention_weights = self.attention(combined)
                weighted_freq = freq_features * attention_weights[:, 0].unsqueeze(1)
                weighted_other = other_features * attention_weights[:, 1].unsqueeze(1)

                # New combined features with attention
                combined_attention = torch.cat([weighted_freq, weighted_other], dim=1)

                # Final processing
                return self.combined_layers(combined_attention)

        # Initialize model
        model = ComponentModel(
            X_train_scaled.shape[1],
            comp_hidden_sizes,
            comp_hyperparams["dropout_rate"],
            freq_indices,
            other_indices,
        ).to(device)

        # Use different loss functions and optimizers for real vs imaginary
        if model_name == "S21_real":
            # Real component is more problematic - use more robust loss
            criterion = nn.SmoothL1Loss(beta=0.05)
            print(f"Using SmoothL1Loss with beta=0.05 for {model_name}")
            optimizer = optim.AdamW(
                model.parameters(),
                lr=comp_hyperparams["learning_rate"],
                weight_decay=comp_hyperparams["weight_decay"],
            )
        else:
            # Imaginary component - custom loss
            def custom_loss(pred, target):
                # Combined loss: 70% Huber, 30% Log-cosh
                huber = nn.HuberLoss(delta=0.1)(pred, target)
                logcosh = log_cosh_loss(pred, target)
                return 0.7 * huber + 0.3 * logcosh

            criterion = custom_loss
            print(f"Using custom combined loss (Huber + Log-cosh) for {model_name}")
            optimizer = optim.Adam(
                model.parameters(),
                lr=comp_hyperparams["learning_rate"],
                weight_decay=comp_hyperparams["weight_decay"],
            )

        # Training loop
        best_val_loss = float("inf")
        best_model_state = None
        patience_counter = 0
        train_losses = []
        val_losses = []

        # Use appropriate scheduler based on component and settings
        if scheduler_type == "cosine_annealing":
            scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
                optimizer, T_max=comp_hyperparams["epochs"], eta_min=1e-6
            )
            print(f"Using CosineAnnealingLR scheduler for {model_name}")
        elif scheduler_type == "one_cycle":
            scheduler = torch.optim.lr_scheduler.OneCycleLR(
                optimizer,
                max_lr=comp_hyperparams["learning_rate"] * 10,
                steps_per_epoch=len(train_loader),
                epochs=comp_hyperparams["epochs"],
            )
            print(f"Using OneCycleLR scheduler for {model_name}")
        else:  # reduce_on_plateau default
            scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
                optimizer, mode="min", factor=0.5, patience=5, verbose=True
            )
            print(f"Using ReduceLROnPlateau scheduler for {model_name}")

        print(f"Starting training for {model_name}...")

        for epoch in range(comp_hyperparams["epochs"]):
            # Training
            model.train()
            running_loss = 0.0

            for inputs, targets in train_loader:
                optimizer.zero_grad()
                outputs = model(inputs)
                loss = criterion(outputs, targets)

                # Add L2 regularization for real component (in addition to optimizer's weight_decay)
                if model_name == "S21_real":
                    l2_reg = 0
                    for name, param in model.named_parameters():
                        if "weight" in name:  # Apply only to weights, not biases
                            l2_reg += torch.norm(param, 2)
                    loss += 1e-5 * l2_reg

                loss.backward()

                # Component-specific gradient clipping
                if model_name == "S21_real":
                    torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=0.5)
                else:
                    torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

                optimizer.step()

                # Update OneCycleLR scheduler if used
                if scheduler_type == "one_cycle":
                    scheduler.step()

                running_loss += loss.item()

            # Calculate average loss
            avg_train_loss = running_loss / len(train_loader)
            train_losses.append(avg_train_loss)

            # Validation
            model.eval()
            with torch.no_grad():
                test_outputs = model(X_test_tensor)
                # Use test labels if available, otherwise just monitor outputs
                if Y_test_comp is not None:
                    Y_test_tensor = torch.FloatTensor(
                        y_scaler.transform(Y_test_comp)
                    ).to(device)
                    val_loss = criterion(test_outputs, Y_test_tensor).item()
                else:
                    val_loss = criterion(
                        test_outputs, torch.zeros_like(test_outputs)
                    ).item()

                val_losses.append(val_loss)

            # Print progress
            if (epoch + 1) % 10 == 0:
                print(
                    f"Epoch {epoch + 1}/{comp_hyperparams['epochs']}, Train Loss: {avg_train_loss:.6f}, Val Loss: {val_loss:.6f}"
                )

            # Update scheduler
            if scheduler_type == "cosine_annealing":
                scheduler.step()
            elif scheduler_type == "reduce_on_plateau":
                scheduler.step(val_loss)
            # one_cycle scheduler is updated after each batch

            # Early stopping
            if val_loss < best_val_loss:
                best_val_loss = val_loss
                best_model_state = model.state_dict().copy()
                patience_counter = 0
            else:
                patience_counter += 1
                if patience_counter >= comp_hyperparams["early_stopping_patience"]:
                    print(f"Early stopping at epoch {epoch + 1}")
                    break

        # Load best model
        if best_model_state is not None:
            model.load_state_dict(best_model_state)

        # Get predictions
        model.eval()
        with torch.no_grad():
            predictions = model(X_test_tensor).cpu().numpy()

        # Inverse transform
        predictions = y_scaler.inverse_transform(predictions)

        # Apply component-specific clipping
        if model_name == "S21_real":
            predictions = np.clip(predictions, real_min, real_max)
            component_idx = 0
        else:
            predictions = np.clip(predictions, imag_min, imag_max)
            component_idx = 1

        # Store in final predictions array
        s21_predictions[:, component_idx] = predictions.flatten()

        # Calculate metrics
        y_true = Y_test[component].values
        y_pred = predictions.flatten()

        metrics = {
            "mse": mean_squared_error(y_true, y_pred),
            "rmse": np.sqrt(mean_squared_error(y_true, y_pred)),
            "r2": r2_score(y_true, y_pred),
            "mae": mean_absolute_error(y_true, y_pred),
            "mape": mean_absolute_percentage_error(y_true, y_pred),
        }

        # Print results
        print(f"\nPerformance metrics for {model_name}:")
        print(f"  RMSE: {metrics['rmse']:.6f}")
        print(f"  R²: {metrics['r2']:.6f}")
        print(f"  MAE: {metrics['mae']:.6f}")
        print(f"  MAPE: {metrics['mape']:.2f}%")

        # Store model and results
        models[model_name] = model
        all_results[model_name] = metrics

        # Plot learning curves
        plt.figure(figsize=(10, 6))
        plt.plot(train_losses, label="Training Loss")
        plt.plot(val_losses, label="Validation Loss")
        plt.xlabel("Epochs")
        plt.ylabel("Loss")
        plt.title(f"Learning Curves for {model_name}")
        plt.legend()
        plt.savefig(f"freq_aware_results/learning_curves_{model_name}.png")
        plt.close()

        # Plot actual vs predicted values
        plt.figure(figsize=(10, 6))
        plt.scatter(y_true, y_pred, alpha=0.3)
        plt.plot([y_true.min(), y_true.max()], [y_true.min(), y_true.max()], "r--")
        plt.xlabel("Actual")
        plt.ylabel("Predicted")
        plt.title(f"Actual vs Predicted for {model_name}")
        plt.savefig(f"freq_aware_results/actual_vs_predicted_{model_name}.png")
        plt.close()

    # Calculate average metrics for S21 (combining real and imaginary)
    s21_metrics = {
        "S_deemb(2,1)_real": all_results["S21_real"],
        "S_deemb(2,1)_imag": all_results["S21_imag"],
    }

    s21_avg_metrics = {
        "rmse": (all_results["S21_real"]["rmse"] + all_results["S21_imag"]["rmse"]) / 2,
        "r2": (all_results["S21_real"]["r2"] + all_results["S21_imag"]["r2"]) / 2,
        "mae": (all_results["S21_real"]["mae"] + all_results["S21_imag"]["mae"]) / 2,
        "mape": (all_results["S21_real"]["mape"] + all_results["S21_imag"]["mape"]) / 2,
    }

    # Print combined results
    print("\nCombined metrics for S21:")
    print(f"  R²: {s21_avg_metrics['r2']:.6f}")
    print(f"  RMSE: {s21_avg_metrics['rmse']:.6f}")
    print(f"  MAE: {s21_avg_metrics['mae']:.6f}")
    print(f"  MAPE: {s21_avg_metrics['mape']:.2f}%")

    # Record total training time
    train_time = time.time() - start_time
    print(f"\nTotal training time: {train_time:.2f} seconds")

    # Save models
    for model_name, model in models.items():
        torch.save(model.state_dict(), f"freq_aware_results/{model_name}_model.pth")

    # Save scalers
    joblib.dump(scalers, "freq_aware_results/s21_component_scalers.pkl")

    print("Models and results saved to freq_aware_results/")

    # The following is needed to match the expected return format
    combined_results = {
        "S21": {"component_metrics": s21_metrics, "avg_metrics": s21_avg_metrics}
    }
    combined_predictions = {"S21": s21_predictions}

    return models, combined_results, combined_predictions, scalers

In [ ]:
def generate_and_save_predictions_as_features(
    models,
    X_train,
    X_test,
    Y_train,
    Y_test,
    scalers=None,
    output_path="freq_aware_results/prediction_features",
    target_sparameter=None,
):
    """
    Generate predictions using trained models and save them as features for training other models.

    Parameters:
    -----------
    models : dict
        Dictionary of trained models from train_frequency_aware_models
    X_train, X_test : pd.DataFrame
        Training and test feature data
    Y_train, Y_test : pd.DataFrame
        Training and test target data
    scalers : dict, optional
        Dictionary of scalers used for Y values during training
    output_path : str
        Path to save the features
    target_sparameter : str, optional
        S-parameter we're targeting next (e.g., 'S12'). Its actual values will be excluded
        to prevent data leakage

    Returns:
    --------
    train_features, test_features : pd.DataFrame
        DataFrames containing prediction features for training and test data
    """
    # Create directory if it doesn't exist
    os.makedirs(output_path, exist_ok=True)

    # Define component mapping for each S-parameter
    component_mapping = {
        "S11": ["S_deemb(1,1)_real", "S_deemb(1,1)_imag"],
        "S12": ["S_deemb(1,2)_real", "S_deemb(1,2)_imag"],
        "S21": ["S_deemb(2,1)_real", "S_deemb(2,1)_imag"],
        "S22": ["S_deemb(2,2)_real", "S_deemb(2,2)_imag"],
    }

    # Initialize DataFrames to store prediction features
    train_features = pd.DataFrame(index=Y_train.index)
    test_features = pd.DataFrame(index=Y_test.index)

    # Determine which components to exclude from actual values to prevent data leakage
    excluded_components = []
    if target_sparameter and target_sparameter in component_mapping:
        excluded_components = component_mapping[target_sparameter]
        print(
            f"Excluding actual values of {target_sparameter} components to prevent data leakage"
        )

    # Check for GPU
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Identify frequency-related features
    freq_indices, other_indices = identify_frequency_features(X_train.columns)

    # Generate predictions for each model
    for model_name, model in models.items():
        print(f"Generating {model_name} predictions...")

        # Get components for this model
        components = component_mapping.get(model_name, [])
        if not components:
            print(f"Warning: Unknown model name {model_name}")
            continue

        # Set model to evaluation mode
        model.eval()

        # Convert input data to tensors
        X_train_tensor = torch.FloatTensor(X_train.values)
        X_test_tensor = torch.FloatTensor(X_test.values)

        # Set feature indices in the model
        model.set_feature_indices(freq_indices, other_indices)

        # Generate predictions
        with torch.no_grad():
            train_preds = model(X_train_tensor.to(device)).cpu().numpy()
            test_preds = model(X_test_tensor.to(device)).cpu().numpy()

        # Apply inverse transform if scaler was used
        if scalers and model_name in scalers and scalers[model_name] is not None:
            train_preds = scalers[model_name].inverse_transform(train_preds)
            test_preds = scalers[model_name].inverse_transform(test_preds)

        # Add predictions as features with appropriate names
        for i, component in enumerate(components):
            feature_name = f"{model_name}_pred_{component}"
            train_features[feature_name] = train_preds[:, i]
            test_features[feature_name] = test_preds[:, i]

    # Save the prediction features
    train_features.to_csv(f"{output_path}/train_prediction_features.csv", index=True)
    test_features.to_csv(f"{output_path}/test_prediction_features.csv", index=True)

    print(f"Saved prediction features to {output_path}")
    print(f"Train features shape: {train_features.shape}")
    print(f"Test features shape: {test_features.shape}")

    return train_features, test_features

In [ ]:
def load_and_combine_prediction_features(
    X_train,
    X_test,
    prediction_path="freq_aware_results/prediction_features",
    append_mode=True,
):
    """
    Load prediction features and combine them with original features.

    Parameters:
    -----------
    X_train, X_test : pd.DataFrame
        Original or previously combined training and test feature data
    prediction_path : str
        Path where prediction features are saved
    append_mode : bool, default=True
        If True, appends new prediction features without overwriting existing ones

    Returns:
    --------
    X_train_combined, X_test_combined : pd.DataFrame
        Combined original and prediction features
    """
    # Load prediction features
    train_predictions = pd.read_csv(
        f"{prediction_path}/train_prediction_features.csv", index_col=0
    )
    test_predictions = pd.read_csv(
        f"{prediction_path}/test_prediction_features.csv", index_col=0
    )

    # Ensure indices match
    train_predictions.index = X_train.index
    test_predictions.index = X_test.index

    # Check for existing prediction columns to avoid duplication
    if append_mode:
        # Get list of new prediction columns
        pred_cols = train_predictions.columns.tolist()

        # Filter out columns that already exist in X_train/X_test
        existing_cols = [col for col in pred_cols if col in X_train.columns]
        new_cols = [col for col in pred_cols if col not in X_train.columns]

        if existing_cols:
            print(
                f"Found {len(existing_cols)} prediction columns already in dataset, preserving existing values"
            )

        if new_cols:
            # Add only new columns to avoid overwriting
            X_train_combined = X_train.copy()
            X_test_combined = X_test.copy()

            for col in new_cols:
                X_train_combined[col] = train_predictions[col]
                X_test_combined[col] = test_predictions[col]

            print(f"Added {len(new_cols)} new prediction columns to existing features")
        else:
            # No new columns to add
            X_train_combined = X_train.copy()
            X_test_combined = X_test.copy()
            print("No new prediction columns to add - all already exist in dataset")
    else:
        # Original behavior - add all columns (potentially overwriting)
        X_train_combined = pd.concat([X_train, train_predictions], axis=1)
        X_test_combined = pd.concat([X_test, test_predictions], axis=1)
        print(
            f"Combined {X_train.shape[1]} original features with {train_predictions.shape[1]} prediction features"
        )

    # Report final feature counts
    print(
        f"Final feature counts - Train: {X_train_combined.shape[1]}, Test: {X_test_combined.shape[1]}"
    )

    return X_train_combined, X_test_combined

## For S(1,1):


In [ ]:
# First, train your S11 model as you've been doing
best_hyperparameters = {
    "learning_rate": 0.001,
    "dropout_rate": 0.1,
    "batch_size": 512,
    "epochs": 200,
    "early_stopping_patience": 30,
    "hidden_sizes": [256, 512, 1024, 512],
    "lr_scheduler_type": "reduce_on_plateau",
    "activation": "gelu",
}

# After training your S11 model
models, results, predictions, scalers = train_frequency_aware_models(
    X_train, X_test, Y_raw_train, Y_raw_test, hyperparameters=best_hyperparameters
)

# Generate S11 predictions to use as features for S12 model
# Specify S12 as the target to ensure we're not leaking its actual values
train_features, test_features = generate_and_save_predictions_as_features(
    models,
    X_train,
    X_test,
    Y_raw_train,
    Y_raw_test,
    scalers=scalers,
    target_sparameter="S21",  # This prevents S12 actual values from being included
)

# Combine with original features
X_train_combined, X_test_combined = load_and_combine_prediction_features(
    X_train, X_test
)

## For S(1,2):


In [ ]:
best_hyperparameters = {
    "hidden_sizes": [384, 768, 1536, 768, 384],
    "dropout_rate": 0.1,
    "learning_rate": 0.002,
    "batch_size": 512,
    "epochs": 300,
    "early_stopping_patience": 40,
    "activation": "gelu",
    "lr_scheduler_type": "reduce_on_plateau",
}

# After training your S11 model
models, results, predictions, scalers = train_frequency_aware_models(
    X_train_combined,
    X_test_combined,
    Y_raw_train,
    Y_raw_test,
    hyperparameters=best_hyperparameters,
)

# Generate S11 predictions to use as features for S12 model
# Specify S12 as the target to ensure we're not leaking its actual values
train_features, test_features = generate_and_save_predictions_as_features(
    models,
    X_train_combined,
    X_test_combined,
    Y_raw_train,
    Y_raw_test,
    scalers=scalers,
    target_sparameter="S21",  # This prevents S12 actual values from being included
)

# Combine with original features
X_train_combined, X_test_combined = load_and_combine_prediction_features(
    X_train_combined, X_test_combined
)

## For S(2,1):


In [ ]:
best_hyperparameters = {
    # Core hyperparameters - keeping stable for now
    "learning_rate": 0.001,
    "dropout_rate": 0.25,
    "batch_size": 256,
    "epochs": 350,  # Slightly increase epochs
    "early_stopping_patience": 40,  # Slightly increase patience
    "activation": "gelu",
    "hidden_sizes": [1024, 512, 256, 128],
    # Architecture improvements for both components
    # S21_real - deeper and wider to improve R²
    "real_hidden_sizes": [2048, 2048, 2048, 1024, 512, 256],
    # S21_imag - improved architecture to reduce MAE
    "imag_hidden_sizes": [2048, 2048, 2048, 2048, 1024, 512],  # Wider and deeper
    # Enhanced branch architectures for both components
    "freq_hidden_sizes": [512, 1024, 2048, 1024],  # Deeper frequency branch
    "other_hidden_sizes": [1024, 2048, 2048, 1024],  # Deeper parameter branch
    # Maintain other parameters
    "weight_decay": 1e-5,
    "real_scheduler": "cosine_annealing",
    "imag_scheduler": "one_cycle",
}

s21_model, s21_metrics, s21_avg_metrics, s21_predictions = train_frequency_aware_models(
    X_train_combined, X_test_combined, Y_raw_train, Y_raw_test, best_hyperparameters
)

## For S(2,2):


In [ ]:
best_hyperparameters = {
    "learning_rate": 0.002,
    "dropout_rate": 0.1,
    "batch_size": 512,
    "epochs": 200,
    "early_stopping_patience": 30,
    "hidden_sizes": [1024, 1536, 2048, 1536, 1024],
    "lr_scheduler_type": "reduce_on_plateau",
    "activation": "gelu",
}

# After training your S11 model
models, results, predictions, scalers = train_frequency_aware_models(
    X_train_combined,
    X_test_combined,
    Y_raw_train,
    Y_raw_test,
    hyperparameters=best_hyperparameters,
)

# Generate S11 predictions to use as features for S12 model
# Specify S12 as the target to ensure we're not leaking its actual values
train_features, test_features = generate_and_save_predictions_as_features(
    models,
    X_train_combined,
    X_test_combined,
    Y_raw_train,
    Y_raw_test,
    scalers=scalers,
    target_sparameter="S21",  # This prevents S12 actual values from being included
)

# Combine with original features
X_train_combined, X_test_combined = load_and_combine_prediction_features(
    X_train_combined, X_test_combined
)